In [1]:
import $ivy.`org.apache.flink::flink-scala:1.15.1`
import $ivy.`org.apache.flink::flink-streaming-scala:1.15.1`
import $ivy.`org.apache.flink::flink-clients:1.14.5`
import $ivy.`org.apache.flink::flink-connector-kafka:1.14.5`

import $ivy.$                                     

import $ivy.$                                               

import $ivy.$                                       

import $ivy.$                                               

In [2]:
import $ivy.`org.apache.logging.log4j:log4j-api:2.18.0`
import $ivy.`org.apache.logging.log4j:log4j-core:2.18.0`

import $ivy.$                                          

import $ivy.$                                           

In [3]:
import org.apache.logging.log4j.{Level, Logger}
import org.apache.flink.api.common.serialization.{DeserializationSchema, SerializationSchema}
import org.apache.flink.api.common.typeinfo.TypeInformation
import org.apache.flink.api.scala._
import org.apache.flink.api.common.state.{AggregatingState, MapState, ValueState, ValueStateDescriptor}
import org.apache.flink.configuration.Configuration
import org.apache.flink.streaming.api.functions.KeyedProcessFunction
import org.apache.flink.util.Collector
import org.apache.flink.api.common.eventtime.WatermarkStrategy
import org.apache.flink.api.scala._
import org.apache.flink.connector.kafka.sink.{KafkaRecordSerializationSchema, KafkaSink}
import org.apache.flink.connector.kafka.source.KafkaSource
import org.apache.flink.connector.kafka.source.enumerator.initializer.OffsetsInitializer
import org.apache.flink.streaming.api.scala.{DataStream, StreamExecutionEnvironment}

import org.apache.logging.log4j.{Level, Logger}

import org.apache.flink.api.common.serialization.{DeserializationSchema, SerializationSchema}

import org.apache.flink.api.common.typeinfo.TypeInformation

import org.apache.flink.api.scala._

import org.apache.flink.api.common.state.{AggregatingState, MapState, ValueState, ValueStateDescriptor}

import org.apache.flink.configuration.Configuration

import org.apache.flink.streaming.api.functions.KeyedProcessFunction

import org.apache.flink.util.Collector

import org.apache.flink.api.common.eventtime.WatermarkStrategy

import org.apache.flink.api.scala._

import org.apache.flink.connector.kafka.sink.{KafkaRecordSerializationSchema, KafkaSink}

import org.apache.flink.connector.kafka.source.KafkaSource

import org.apache.flink.connector.kafka.source.enumerator.initializer.OffsetsInitializer

import org.apache.flink.streaming.api.scala.{DataStream, StreamExecutionEnvironment}

In [4]:
object Transaction {

  case class Transaction(id: Int, amount: Int, transaction_type: String)

  // Deserialize Value from Source
  class TransactionValueDeserializer extends DeserializationSchema[Transaction] {
    override def deserialize(messageAsByte: Array[Byte]): Transaction = {
      val message: String = new String(messageAsByte)
      val tokens: Array[String] = message.split(",")
      val id: Int = tokens(0).toInt
      val amount: Int = tokens(1).toInt
      val transaction_type: String = tokens(2)
      Transaction(id, amount, transaction_type)
    }

    override def isEndOfStream(nextElement: Transaction): Boolean = false

    override def getProducedType: TypeInformation[Transaction] = implicitly[TypeInformation[Transaction]]
  }

  // Serialize Key to Sink
  class TransactionKeySerializer extends SerializationSchema[Transaction] {
    override def serialize(key: Transaction): Array[Byte] =
      s"${key.id}".getBytes()
  }

  // Serialize Value to Sink
  class TransactionValueSerializer extends SerializationSchema[Transaction] {
    override def serialize(transaction: Transaction): Array[Byte] =
      s"${transaction.id},${transaction.amount},${transaction.transaction_type}".getBytes("UTF-8")
  }
}

defined object Transaction

In [5]:
import Transaction._

import Transaction._

In [6]:
class FraudProcessor extends KeyedProcessFunction[Int, Transaction, Transaction] {
  var stateIDMapping: MapState[String, Any] = _
  var countValue: ValueState[Long] = _
  var aggAvg: AggregatingState[Int, Double] = _


  @throws[Exception]
  override def open(parameters: Configuration): Unit = {
    countValue = getRuntimeContext.getState(
      new ValueStateDescriptor[Long]("events-counter", classOf[Long])
    )
  }

  // Logic for each Key
  override def processElement(transaction: Transaction,
                              ctx: KeyedProcessFunction[Int, Transaction, Transaction]#Context,
                              out: Collector[Transaction]): Unit = {

    // Handle the State here
    val nEventsForThisUserID = countValue.value()
    countValue.update(nEventsForThisUserID + 1)

    println(s"${transaction.id} with Count ${countValue.value()}")

    out.collect(transaction)
  }
}

defined class FraudProcessor

In [7]:
val env: StreamExecutionEnvironment = StreamExecutionEnvironment.getExecutionEnvironment

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


env: StreamExecutionEnvironment = org.apache.flink.streaming.api.scala.StreamExecutionEnvironment@331e7973

In [8]:
// Sink
val kafkaSink: KafkaSink[Transaction] = KafkaSink.builder[Transaction]()
  .setBootstrapServers("localhost:9092")
  .setRecordSerializer(KafkaRecordSerializationSchema.builder()
    .setTopic("potential_fraud_transactions")
    .setValueSerializationSchema(new TransactionValueSerializer)
    .setKeySerializationSchema(new TransactionKeySerializer).build()
  ).build()

kafkaSink: KafkaSink[Transaction] = org.apache.flink.connector.kafka.sink.KafkaSink@e92149a

In [9]:
val kafkaSource: KafkaSource[Transaction] = KafkaSource.builder[Transaction]()
  .setBootstrapServers("localhost:9092")
  .setTopics("transactions")
  .setGroupId("events-group")
  .setStartingOffsets(OffsetsInitializer.earliest())
  .setValueOnlyDeserializer(new TransactionValueDeserializer)
  .build()

kafkaSource: KafkaSource[Transaction] = org.apache.flink.connector.kafka.source.KafkaSource@2aca9a1b

In [10]:
val transactionStream: DataStream[Transaction] = env.fromSource(kafkaSource,
  WatermarkStrategy.noWatermarks(),
  "Kafka Source")

transactionStream: DataStream[Transaction] = org.apache.flink.streaming.api.scala.DataStream@606741e1

In [11]:
val filteredTransactionStream: DataStream[Transaction] = transactionStream
  .keyBy(transaction => transaction.id)
  .process(new FraudProcessor)

filteredTransactionStream: DataStream[Transaction] = org.apache.flink.streaming.api.scala.DataStream@25422572

In [12]:
filteredTransactionStream.sinkTo(kafkaSink)

res11: org.apache.flink.streaming.api.datastream.DataStreamSink[Transaction] = org.apache.flink.streaming.api.datastream.DataStreamSink@20012880

In [13]:
filteredTransactionStream.print()

res12: org.apache.flink.streaming.api.datastream.DataStreamSink[Transaction] = org.apache.flink.streaming.api.datastream.DataStreamSink@63cf4b4

In [14]:
env.execute()

16 with Count 1
2 with Count 1
25 with Count 1
10 with Count 1
13 with Count 1
8 with Count 1
18 with Count 1
17 with Count 1
14 with Count 1
28 with Count 1
27 with Count 1
12 with Count 1
11 with Count 1
16> Transaction(2,35,debit)
6> Transaction(25,16,credit)
1> Transaction(13,64,debit)
12> Transaction(27,89,credit)
13> Transaction(11,54,debit)
11 with Count 2
11> Transaction(17,33,debit)
26 with Count 1
15> Transaction(16,69,credit)
16 with Count 2
10> Transaction(18,53,debit)
18 with Count 2
4> Transaction(14,22,debit)
2> Transaction(8,51,credit)
14 with Count 2
10> Transaction(18,17,debit)
13> Transaction(11,9,debit)
7> Transaction(12,61,debit)
9 with Count 1
15> Transaction(16,4,credit)
7> Transaction(9,73,debit)
3> Transaction(28,9,credit)
16> Transaction(26,83,credit)
9> Transaction(10,37,debit)
1 with Count 1
27 with Count 2
13 with Count 2
22 with Count 1
6> Transaction(22,35,debit)
22 with Count 2
1> Transaction(13,77,credit)
12> Transaction(27,5,credit)
11> Transaction(1,7

: 